# Swissmetro test

In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
import pandas as pd
import pycmtensor as cmt

sm = pd.read_csv("../data/swissmetro.dat", sep="\t")
db = cmt.Data(
    df=sm,
    choice="CHOICE",
    drop=[sm["CHOICE"] == 0],
    autoscale=True,
    autoscale_except=["ID", "ORIGIN", "DEST", "CHOICE"],
    split=0.8,
)

In [ ]:
from pycmtensor.expressions import Beta
from pycmtensor.models import MNL

b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
asc_train = Beta("asc_train", 0.0, None, None, 0)
asc_car = Beta("asc_car", 0.0, None, None, 0)
asc_sm = Beta("asc_sm", 0.0, None, None, 1)

U_1 = b_cost * db["TRAIN_CO"] + b_time * db["TRAIN_TT"] + asc_train
U_2 = b_cost * db["SM_CO"] + b_time * db["SM_TT"] + asc_sm
U_3 = b_cost * db["CAR_CO"] + b_time * db["CAR_TT"] + asc_car

# specify the utility function and the availability conditions
U = [U_1, U_2, U_3]  # utility
AV = [db["TRAIN_AV"], db["SM_AV"], db["CAR_AV"]]  # availability

mymodel = MNL(db, locals(), U, AV, name="MNL")
mymodel.config.set_hyperparameter(max_steps=300)

Model training

In [ ]:
mymodel.train(db)

Print results and statistics

Beta parameters

In [ ]:
np.set_printoptions(suppress=True)
CR = cmt.statistics.variance_covariance(mymodel.results.hessian_matrix)
CR

In [ ]:
BHHH = cmt.statistics.variance_covariance(-mymodel.results.bhhh_matrix)
BHHH

In [ ]:
CR.dot(mymodel.results.bhhh_matrix.dot(CR))

In [ ]:
print(mymodel.results.beta_statistics())

Model statistics

In [ ]:
print(mymodel.results.model_statistics())

Correlation matrix

In [ ]:
print(mymodel.results.model_correlation_matrix())

Benchmark

In [ ]:
print(mymodel.results.benchmark())

Predictions

In [ ]:
# predictions
print(mymodel.predict(db, return_choices=False))
print(np.unique(mymodel.predict(db), return_counts=True))

Elasticities

In [ ]:
from pycmtensor.statistics import elasticities

print(elasticities(mymodel, db, 0, "TRAIN_TT"))